In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd

In [3]:
sc = SparkContext('local')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/20 05:59:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/20 05:59:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
sqlCtx = SQLContext(sc)

/root/main/spark-3.3.1-bin-hadoop2/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [7]:
emp = [
    ('홍길동',1),
    ('이순신',2),
    ('임꺽정',3),
    ('김철수',4),
    ('호날두',5)
]
dept = [
    ('개발',1),
    ('연구',2),
    ('영업',3),
    ('기획',4)
]

In [9]:
emprdd = sc.parallelize(emp)
emprdd.take(3)

[('홍길동', 1), ('이순신', 2), ('임꺽정', 3)]

In [10]:
deptrdd = sc.parallelize(dept)
deptrdd.take(3)

[('개발', 1), ('연구', 2), ('영업', 3)]

In [11]:
# rdd => spark dataframe
empdf1=emprdd.toDF()

In [14]:
type(empdf1)

pyspark.sql.dataframe.DataFrame

In [13]:
empdf1.show() # rdd 의 take 같은 느낌

+------+---+
|    _1| _2|
+------+---+
|홍길동|  1|
|이순신|  2|
|임꺽정|  3|
|김철수|  4|
|호날두|  5|
+------+---+



In [15]:
empdf1.toPandas() # rdd 의 collect랑 같은 느낌

,_1,_2
0,홍길동,1
1,이순신,2
2,임꺽정,3
3,김철수,4
4,호날두,5


In [17]:
empdf2 = sqlCtx.createDataFrame( emp )

In [21]:
empdf2.show()

+------+---+
|    _1| _2|
+------+---+
|홍길동|  1|
|이순신|  2|
|임꺽정|  3|
|김철수|  4|
|호날두|  5|
+------+---+



In [22]:
empdf2 = sqlCtx.createDataFrame( emp ,['name','deptid'])
empdf2.show()

+------+------+
|  name|deptid|
+------+------+
|홍길동|     1|
|이순신|     2|
|임꺽정|     3|
|김철수|     4|
|호날두|     5|
+------+------+



In [ ]:
pd.read_csv(thousands=)

In [44]:
seoulDf = sqlCtx.read.csv('./main/data/crime_in_Seoul.csv',encoding='cp949', 
                          header=True,
                          inferSchema=True,
                          )
seoulDf.show()

TypeError: csv() got an unexpected keyword argument 'thousands'

In [43]:
seoulDf.printSchema()

root
 |-- 관서명: string (nullable = true)
 |-- 살인 발생: integer (nullable = true)
 |-- 살인 검거: integer (nullable = true)
 |-- 강도 발생: integer (nullable = true)
 |-- 강도 검거: integer (nullable = true)
 |-- 강간 발생: integer (nullable = true)
 |-- 강간 검거: integer (nullable = true)
 |-- 절도 발생: string (nullable = true)
 |-- 절도 검거: string (nullable = true)
 |-- 폭력 발생: string (nullable = true)
 |-- 폭력 검거: string (nullable = true)



In [ ]:
# to_numeric

In [45]:
empdf2.createOrReplaceTempView('myemp')

In [48]:
sql =  'select * from myemp'
sqlDF=sqlCtx.sql(sql)
sqlDF.show()

In [50]:
sql =  'select name from myemp'
sqlDF=sqlCtx.sql(sql)
sqlDF.show()

+------+
|  name|
+------+
|홍길동|
|이순신|
|임꺽정|
|김철수|
|호날두|
+------+



In [52]:
sql =  'select * from myemp from where deptid=2'
sqlDF=sqlCtx.sql(sql)
sqlDF.show()

+------+------+
|  name|deptid|
+------+------+
|이순신|     2|
+------+------+



In [53]:
sql =  'select * from myemp from where deptid=2 or deptid=3'
sqlDF=sqlCtx.sql(sql)
sqlDF.show()

+------+------+
|  name|deptid|
+------+------+
|이순신|     2|
|임꺽정|     3|
+------+------+



In [63]:
data1 = [
    '홍길동,1000',
    '이순신,2000',
    '임꺽정,3000',
    '김철수,4000',
    '이황,5000',
    '이이,6000',

]
# sparkdataframe 변환후 아래 문제를 spark sql로 작성

In [58]:
main = sc.parallelize(data1)
main_df=main.toDF()

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd
sc = SparkContext('local')
sqlCtx = SQLContext(sc)

In [96]:

data1 = [
    ('홍길동,1000'),
    ('이순신,2000'),
    ('임꺽정,3000'),
    ('김철수,4000'),
    ('이황,5000'),
    ('이이,6000'),

]
# sparkdataframe 변환후 아래 문제를 spark sql로 작성

In [97]:
main = sc.parallelize(data1)
main_df = sqlCtx.createDataFrame( main.map(lambda x: (x.split(',')[0], int(x.split(',')[1]))) ,['name','salary'],)

In [98]:
main_df.show()

+------+------+
|  name|salary|
+------+------+
|홍길동|  1000|
|이순신|  2000|
|임꺽정|  3000|
|김철수|  4000|
|  이황|  5000|
|  이이|  6000|
+------+------+



In [99]:
main_df.createOrReplaceTempView('main_df')

In [100]:
# prob_1
sql =  'SELECT * FROM main_df WHERE SALARY = (SELECT MAX(SALARY) FROM main_df)'
sqlDF=sqlCtx.sql(sql)
sqlDF.show()

+----+------+
|name|salary|
+----+------+
|이이|  6000|
+----+------+



In [101]:
# prob_2
sql =  'SELECT *, (salary-salary*0.033) AS TAX FROM main_df '
sqlDF=sqlCtx.sql(sql)
sqlDF.show()

+------+------+--------+
|  name|salary|     TAX|
+------+------+--------+
|홍길동|  1000| 967.000|
|이순신|  2000|1934.000|
|임꺽정|  3000|2901.000|
|김철수|  4000|3868.000|
|  이황|  5000|4835.000|
|  이이|  6000|5802.000|
+------+------+--------+



In [102]:
# prob_3
sql =  'SELECT *  FROM main_df WHERE NAME LIKE ("%철%") '
sqlDF=sqlCtx.sql(sql)
sqlDF.show()

+------+------+
|  name|salary|
+------+------+
|김철수|  4000|
+------+------+



In [103]:
# prob_4
sql =  'SELECT *  FROM main_df ORDER BY SALARY DESC LIMIT 5'
sqlDF=sqlCtx.sql(sql)
sqlDF.show()

+------+------+
|  name|salary|
+------+------+
|  이이|  6000|
|  이황|  5000|
|김철수|  4000|
|임꺽정|  3000|
|이순신|  2000|
+------+------+



In [104]:
# prob_5
sql =  'SELECT *  FROM main_df WHERE SALARY BETWEEN 2000 AND 4000'
sqlDF=sqlCtx.sql(sql)
sqlDF.show()

+------+------+
|  name|salary|
+------+------+
|이순신|  2000|
|임꺽정|  3000|
|김철수|  4000|
+------+------+



In [105]:
# prob_6
# 임금의 최댓값과 최솟값의 차이를 구하시오
sql =  'SELECT MAX(SALARY)-MIN(SALARY) AS DIFF_SALARY  FROM main_df'
sqlDF=sqlCtx.sql(sql)
sqlDF.show()

+-----------+
|DIFF_SALARY|
+-----------+
|       5000|
+-----------+



In [108]:
# prob_7
# 임금의 표준편차를 구하시오 단 (db는 표본집단임) 구하시오
sql =  'SELECT STD(SALARY)*(COUNT(*)/(COUNT(*)-1)) AS STD_SALARY  FROM main_df'
sqlDF=sqlCtx.sql(sql)
sqlDF.show()

+------------------+
|        STD_SALARY|
+------------------+
|2244.9944320643644|
+------------------+



In [109]:
# prob_8
# 최소 임금을 받는 데이터 출력
sql = 'select name,salary from main_df order by salary limit 1'
sqlDF = sqlCtx.sql(sql)
sqlDF.show()

+------+------+
|  name|salary|
+------+------+
|홍길동|  1000|
+------+------+



In [110]:
# prob_9
#글자수가 2 이하인 데이터 출력
sql = 'select name,salary from main_df where length(name)<3'
sqlDF = sqlCtx.sql(sql)
sqlDF.show()

+----+------+
|name|salary|
+----+------+
|이황|  5000|
|이이|  6000|
+----+------+



In [111]:
# prob_10
#이로 시작하는 사람들의 임금합
sql = "select sum(salary) from main_df where name like '이%'"
sqlDF = sqlCtx.sql(sql)
sqlDF.show()

+-----------+
|sum(salary)|
+-----------+
|      13000|
+-----------+

